## Overview

* Relevant Chapter: 1.1
* This notebook introduces the concept of complex functions. Note that when plotting a complex function it cannot be plotted as a surface plot such as f(x, y), since there are two components of a complex number, either when the output is plotted in polar form or cartesian form.

In [1]:
%pylab inline
import sympy
sympy.init_printing()

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [28]:
def plot_complex_function(Y, sigma_min, sigma_max, omega_max, log_omega, plot_conj):
    sigma_eval=np.linspace(sigma_min, sigma_max, 200)
    
    # constants
    omega_min = 1e-3
    nlevels = 20  # levels of contour
    nlevels_outline = 10  # levels of contour

    # validate input
    assert np.all(omega_max > 0)
    
    # symbolic function
    sigma, omega = sympy.symbols('sigma, omega', real=True, positive=True)
    s_binomial = sigma + sympy.I*omega
    Y_arg = sympy.arg(Y.subs(s, s_binomial))
    Y_mag = sympy.Abs(Y.subs(s, s_binomial))
    f_Y_euler = sympy.lambdify((sigma, omega), (Y_arg, Y_mag))
    
    # compute poles and zeros
    poles = sympy.solve(sympy.denom(Y))
    zeros = sympy.solve(sympy.numer(Y))
    print(np.real(poles))
    
    re_poles = np.array(poles.as_real_imag()[0], dtype=float)
    im_poles = np.array(poles.as_real_imag()[1], dtype=float)
    re_zeros = np.array(zeros.as_real_imag()[0], dtype=float)
    im_zeros = np.array(zeros.as_real_imag()[1], dtype=float)

    # evaluate
    if log_omega:
        omega_eval=np.logspace(np.log10(omega_min), np.log10(omega_max), 200)
        yplot = plt.semilogy
        xplot = plt.semilogx
        im_zeros = np.where(im_zeros < omega_min, omega_min, im_zeros)
        im_poles = np.where(im_poles < omega_min, omega_min, im_poles)
    else:
        omega_eval=np.linspace(omega_min, omega_max, 100)
        omega_both = np.concatenate([-np.flip(omega_eval), omega_eval])
        yplot = plt.plot
        xplot = plt.plot
    
    if plot_conj and not log_omega:
        Sigma, Omega = np.meshgrid(sigma_eval, omega_both)
    else:
        Sigma, Omega = np.meshgrid(sigma_eval, omega_eval)

    Z = f_Y_euler(Sigma, Omega)
    angle = np.rad2deg(Z[0])
    mag = 20*np.log10(np.abs(Z[1]))
    
    # angle G(jw)
    Z_jw = f_Y_euler(0, omega_eval)
    angle_jw = np.rad2deg(Z_jw[0])
    mag_jw = 20*np.log10(np.abs(Z_jw[1]))
    
    # figures
    fig = plt.figure(figsize=(20, 10))
    
    # angle of G(S)
    ax = plt.subplot(221)
    CS = ax.contourf(Sigma, Omega, angle, nlevels,cmap=plt.cm.hsv)
    cbar = plt.colorbar(CS)
    CS = ax.contour(Sigma, Omega, angle, nlevels_outline, colors='k', linestyles='--')
    cbar.add_lines(CS)
    ax.set_title(r'$\angle G(s),\ deg$');
    ax.set_xlabel(r'$\sigma$')
    ax.set_ylabel(r'$\omega,\ rad/s$')
    pole_props = {'marker':'x', 'color':'white', 'markeredgewidth':3, 'markersize':20, 'markerfacecolor':'none'}
    zero_props = {'marker':'o', 'color':'white', 'markeredgewidth':3, 'markersize':20, 'markerfacecolor':'none'}
    h_pole = yplot(re_poles, im_poles, **pole_props)
    h_zero = yplot(re_zeros, im_zeros, **zero_props)
    h_jw = yplot(0*omega_eval, omega_eval, 'g-', linewidth=5, alpha=0.5)
    plt.legend([h_zero[0], h_pole[0], h_jw[0]], ['zero', 'pole', 'jw'], loc='best', ncol=3)
    plt.grid()
    
    # mag of G(S)
    ax = plt.subplot(222)
    CS2 = ax.contourf(Sigma, Omega, mag, nlevels, cmap=plt.cm.coolwarm)
    cbar = plt.colorbar(CS2)
    CS2 = ax.contour(Sigma, Omega, mag, nlevels_outline, colors='k', linestyles='--')
    cbar.add_lines(CS2)
    ax.set_title(r'$|G(s)|,\ dB$');
    ax.set_xlabel(r'$\sigma$')
    ax.set_ylabel(r'$\omega,\ rad/s$')
    h_pole = yplot(re_poles, im_poles, **pole_props)
    h_zero = yplot(re_zeros, im_zeros, **zero_props)
    h_jw = yplot(0*omega_eval, omega_eval, 'g-', linewidth=5, alpha=0.5)
    plt.legend([h_zero[0], h_pole[0], h_jw[0]], ['zero', 'pole', 'jw'], loc='best', ncol=3)
    plt.grid()
    
    # magnitude of G(jw)
    ax = plt.subplot(223)
    ax.set_title(r'$\angle G(j\omega),\ deg$');
    ax.set_xlabel(r'$\omega,\ rad/s$')
    xplot(omega_eval, angle_jw)
    xplot(omega_eval, angle_jw)
    ax.set_ylabel(r'$\angle G(j\omega),\ deg$');
    plt.grid()

    ax = plt.subplot(224)
    ax.set_title(r'$|G(j\omega)|,\ dB$');
    ax.set_xlabel(r'$\omega,\ rad/s$')
    xplot(omega_eval, mag_jw)
    xplot(omega_eval, mag_jw)
    ax.set_ylabel(r'$|G(j\omega)|,\ dB$');
    plt.grid()

In [30]:
t, sigma, omgega = sympy.symbols('t, sigma, omega')
s = sympy.symbols('s', complex=True)

g = 9.8
l = 1.6
zeta = 0.5
wn = np.sqrt(g/l)
G = wn**2/(s**2 + 2*zeta*wn*s + wn**2)
G

In [31]:
plot_complex_function(G, sigma_min=-5, sigma_max=1, omega_max=100, log_omega=True, plot_conj=True)

[-1.23743686707646 - 2.14330352493528*I
 -1.23743686707646 + 2.14330352493528*I]


AttributeError: 'list' object has no attribute 'as_real_imag'

In [14]:
mag = 10
mag_dB = 20*np.log10(mag)
mag_dB